In [1]:
#packages required 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

In [2]:
# website to be scraped
url="https://www.coingecko.com/en/exchanges"

In [3]:
#Requirement : Chrome driver should be installed in the same directory in which code is running
driver = webdriver.Chrome() #This code will show error if chrome driver is not installed
driver.get(url)

In [4]:
#Scraping table from the data
data=pd.read_html(driver.page_source)

In [5]:
final_data=data[0]

In [6]:
#Cleaning data
#Exchange columne
df = pd.DataFrame(data[0].Exchange.str.split(' ').tolist())
df['Exchange_name']=' '
df['Exchange_name'][df[1]!=' ']=df[0]+df[1]
df['Exchange_name'][~df[2].isin(['Centralized','','Decentralized'])]=df['Exchange_name']+df[2]
df['Centralised_Decentralised']=''
df['Centralised_Decentralised'][df[1].isin(['Centralized','Decentralized'])]=df[1]
df['Centralised_Decentralised'][df[2].isin(['Centralized','Decentralized'])]=df[2]
df['Centralised_Decentralised'][df[3].isin(['Centralized','Decentralized'])]=df[3]
df['Centralised_Decentralised'][df[4].isin(['Centralized','Decentralized'])]=df[4]
final_data['Exchange_name']=df['Exchange_name']
final_data['Centralised_Decentralised']=df['Centralised_Decentralised']
#Trading incentive
df = pd.DataFrame(data[0]['24h Volume'].str.split(' ',1).tolist())
final_data['24h Volume_amt']=df[0]
final_data['Trading_incentive']=df[1]
#Most Active Traded
bcd=[]
for val in data[0]['Most Actively Traded'].str.split('$').tolist():
    if type(val)==type(1.1):
        bcd.append(['NA','NA'])
    else:
        bcd.append(val)
df=pd.DataFrame(bcd)
final_data['Most Actively Traded_currency']=df[0]
final_data['value_in_dollars']=df[1]
#droping unnecessary columns
final_data1=final_data.drop(['Exchange','24h Volume','Most Actively Traded'],axis=1)

In [7]:
final_data1.head()

,#,# Coins,# Pairs,Country,Established,Last 7 Days,Twitter Followers,Alexa Rank,Exchange_name,Centralised_Decentralised,24h Volume_amt,Trading_incentive,Most Actively Traded_currency,value_in_dollars
0,1,44.0,106.0,NaN,2018.0,NaN,13698.0,#49842,BitMax,Centralized,"$4,391,980,778",Trading Incentives,PAX/USDT,"2,480,241,470"
1,2,117.0,209.0,NaN,2018.0,NaN,4723.0,#75693,FCoin,Centralized,"$2,530,316,892",Trading Incentives,BTC/USDT,"1,397,999,857"
2,3,51.0,119.0,NaN,2018.0,NaN,1959.0,#7608,ZBG,Centralized,"$1,661,587,980",Trading Incentives,HMK/QC,0.00000000
3,4,164.0,513.0,Belize,2013.0,NaN,183538.0,#5388,OKEx,Centralized,"$1,568,881,249",Trading Incentives,BTC/USDT,"358,280,169"
4,5,78.0,219.0,China,2017.0,NaN,28180.0,#19361,ZB,Centralized,"$1,524,845,457",Trading Incentives,BTC/USDT,"205,941,219"


In [8]:
# filtering data on the basis of country and Centralised/Decentralised column
final_data_filtered=final_data1[(final_data1['Country'].isin(['Australia','Croatia','Cyprus','Czech Republic',\
                                                              'Estonia','Hong Kong','Japan','Ireland','Malta','Netherlands','Norway',\
                                                             'Singapore','Poland','South Korea','Switzerland','United Kingdom','United States']))
                                &(final_data1['Centralised_Decentralised'].isin(['Centralized']))]

In [9]:
final_data_filtered.head()

,#,# Coins,# Pairs,Country,Established,Last 7 Days,Twitter Followers,Alexa Rank,Exchange_name,Centralised_Decentralised,24h Volume_amt,Trading_incentive,Most Actively Traded_currency,value_in_dollars
5,6,150.0,463.0,Malta,2017.0,NaN,952594.0,#1003,Binance,Centralized,"$1,422,066,858",None,BTC/USDT,"345,959,692"
6,7,37.0,111.0,Australia,2017.0,NaN,NaN,#11623,BW.com,Centralized,"$1,285,586,022",None,BTC/USDT,"581,175,158"
11,12,27.0,70.0,Australia,2018.0,NaN,1034.0,-,BTCEXA,Centralized,"$988,784,141",None,BTC/USDT,"305,170,247"
13,14,163.0,208.0,Singapore,2017.0,NaN,11483.0,#47015,CoinBene,Centralized,"$947,394,332",Trading Incentives,BTC/USDT,"563,159,529"
20,21,80.0,146.0,Singapore,2017.0,NaN,21869.0,#38491,CoinTiger,Centralized,"$825,812,470",None,BTC/USDT,"214,258,133"
